## Setup

### Import TensorFlow and other libraries

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import time

import numpy as np
import tensorflow as tf

### Download the Shakespeare dataset

In [3]:
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)

### Read the data

In [4]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


Let's take a look at the first 250 characters in text

In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Let's check to see how many unique characters are in our corpus/documen

In [6]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

65 unique characters


## Process the text

### Vectorize the text

Before training, we need to convert the strings to a numerical representation.

Using tf.keras.layers.StringLookup layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ["abcdefg", "xyz"]

# TODO 1
chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Create the tf.keras.layers.StringLookup layer:

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

It converts from tokens to character IDs:

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Important to invert this representation and recover human-readable strings from it. For this you can use tf.keras.layers.StringLookup(..., invert=True).

Note: Here instead of passing the original vocabulary generated with sorted(set(text)) use the get_vocabulary() method of the tf.keras.layers.StringLookup layer so that the [UNK] tokens is set the same way.

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

This layer recovers the characters from the vectors of IDs, and returns them as a tf.RaggedTensor of characters:

A RaggedTensor is a type of tensor in TensorFlow that can hold variable-length sequences — like sentences or words — without needing to pad them to the same length. This is crucial for text generation since each batch of text might produce outputs of varying lengths.

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

tf.strings.reduce_join to join the characters back into strings.

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we are training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

First use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [14]:
# TODO 2
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

Creates a `tf.data.Dataset` where each item is a single ID (one per character).

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i
t
i


In [17]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)
examples_per_epoch

11043

The `batch` method easily convert these individual characters to sequences of the desired size.

In [18]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [19]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Create training batches

Used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

A model with the following layers

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units`.
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [25]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

The class below does the following:
- We derive a class from tf.keras.Model
- The constructor is used to define the layers of the model
- We define the pass forward using the layers defined in the constructor

In [26]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        # TODO - Create an embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # TODO - Create a GRU layer
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        # TODO - Finally connect it with a dense layer
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        # since we are training a text generation model,
        # we use the previous state, in training. If there is no state,
        # then we initialize the state
        if states is None:
            states = [tf.zeros((tf.shape(inputs)[0], self.gru.units))]
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character.

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [27]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 100, 256)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ((64, 100, 1024), (64,      │       3,938,304 │
│                                      │ 1024))                      │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 100, 66)               │          67,650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Trying it for the first example in the batch:

In [30]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0], num_samples=1
)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [31]:
sampled_indices

array([28, 64, 38, 54, 10, 20, 40, 10, 15, 48, 52, 56, 14, 65, 60, 34, 28,
       45, 15,  9, 61, 31, 58, 15, 14, 45, 53, 34, 63, 48, 26, 58, 24, 36,
        1,  4, 62, 53,  9, 27, 44,  3, 18, 52,  1, 19, 53, 54, 45, 33,  1,
        9, 32, 37, 47, 51, 53, 51, 57,  3, 43,  3, 52, 11, 58, 40, 45, 63,
       58, 51, 14, 43, 37, 50, 10, 45, 27, 15, 50, 43,  0, 50, 42,  5, 56,
       57, 11, 54, 27,  7, 40, 36, 15, 63, 60, 23, 51, 23,  1, 36])

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' Cominius\nWith thee awhile: determine on some course,\nMore than a wild exposture to each chance\nThat'

Next Char Predictions:
 b'OyYo3Ga3BimqAzuUOfB.vRsBAfnUxiMsKW\n$wn.Ne!Em\nFnofT\n.SXhlnlr!d!m:safxslAdXk3fNBkd[UNK]kc&qr:oN,aWBxuJlJ\nW'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

#### Attach an optimizer, and a loss function
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.

In [33]:
# Add a loss function here
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190694, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this we can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [35]:
tf.exp(example_batch_mean_loss).numpy()

np.float32(66.06862)

Configure the training procedure using the `tf.keras.Model.compile` method. Using `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [36]:
model.compile(optimizer="adam", loss=loss)

### Configure checkpoints

Using a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model.

In [38]:
EPOCHS = 10

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 456s 3s/step - loss: 3.0430
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 441s 3s/step - loss: 1.9070
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 437s 3s/step - loss: 1.6233
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 440s 3s/step - loss: 1.4756
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 446s 3s/step - loss: 1.3880
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 447s 3s/step - loss: 1.3282
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - loss: 1.2764
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 409s 2s/step - loss: 1.2315
Epoch 9/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 445s 3s/step - loss: 1.1880
Epoch 10/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 446s 3s/step - loss: 1.1477


### Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.

The following makes a single step prediction:

In [46]:
from keras.saving import register_keras_serializable

@register_keras_serializable(package="Custom")
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # If components are not None, create the mask
        if self.ids_from_chars is not None:
            skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
            sparse_mask = tf.SparseTensor(
                values=[-float("inf")] * len(skip_ids),
                indices=skip_ids,
                dense_shape=[len(ids_from_chars.get_vocabulary())]
            )
            self.prediction_mask = tf.sparse.to_dense(sparse_mask)
        else:
            self.prediction_mask = None

    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature

        if self.prediction_mask is not None:
            predicted_logits = predicted_logits + self.prediction_mask

        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        predicted_chars = self.chars_from_ids(predicted_ids)

        return predicted_chars, states

    def get_config(self):
        return {
            "temperature": self.temperature
        }

    @classmethod
    def from_config(cls, config):
        # Initialize with placeholders; real components reattached after loading
        return cls(model=None, chars_from_ids=None, ids_from_chars=None, **config)

    def reattach_components(self, model, chars_from_ids, ids_from_chars):
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Recreate the mask
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())]
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

In [48]:
one_step_model = OneStep(model= model, chars_from_ids=chars_from_ids, ids_from_chars=ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, we'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [50]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
Prithee, nuns!

JULIET:
I pray thy royal and awaking are about,
Because those that misposticulish knowledge
Oft to our queen and unrest
only; nay, look up in prisoner. I till hemp your lordship
That my hands have been thy footless armice you,
When with myse fine or enough that through allow'd that prevail
What concerd id the whiles, looks on thee,
But in thy rests that shall be thusbed with me;
'Tis no singland cannib.
But, Hortensio, go with veint calmmed! Hence with the view of
them that respect his subthen can win a doubzed of
cheer yourself.
Most pleasing yourself in tow. Your houses!
Let me see them not such seen to stand.

PETRUCHIO:
I will, that now mine enemies,
sir, about a disconder further than they
Within light. We do not sin,
That lies a hopel on the shoes amended,
Or else safelety of a croldage. He's further
As I have let him in his defend.

Surse:
Right.

WARWICK:
O Blarance.

ANTIGONUS:
Who seeman's death, and what lith,
I did suppose yourself.

AUTOLYCUS:
It is 

If we want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [52]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
for s in result:
    print(s.numpy().decode("utf-8"))
print("\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
But he hath patience!
I enjoy no crush: I have spoke, live,
Or as they have a second fates,
There you to cove on thee; Overfore.
That lives. I do speak thee eyes
Or else dready temples; flies shall go to you.
You are thy dangerous provinces!

BRUTUS:
Bear the English crown and stay up
not then, not forth the sanctuary;
Or else have fies' out wine and pays her;
Though o'er the names come disposed, would adjumisue
That word more wrathful money, may stand
The allibour: therefore fares my wife and death.

YORK:
Ronebrother, welcome done, and were arr from this land:
Knock the noise of the thoughts will have more varit more and old
too much seat; no warriors wakes,
The son--hath poisoned heaven and were none,
Have a duley apear or no.

DUKE OF YORK:
It must your will and dut some confererity
often enforced to chyself?

POLIXENES:
Displate me! that's the gods among?

BRUTUS:
Said upon the thing is called much.

ISABELLA:
I pray; the love I never shall have beared my;
And while the dru

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing us to use it anywhere a `tf.saved_model` is accepted.

In [54]:
# Save the OneStep model
one_step_model.save('one_step_model.keras')

# Load the OneStep model
one_step_reloaded = tf.keras.models.load_model(
    'one_step_model.keras',
    custom_objects={'OneStep': OneStep}
)

# Reattach components
one_step_reloaded.reattach_components(model, chars_from_ids, ids_from_chars)

In [114]:
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Or brain! which, doop! take my what lave we say poison,
Which we parcillow us the state?
What? can 
